<a href="https://colab.research.google.com/github/shrisrivas/google-co_lab_prog/blob/main/cuda_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Sep 15 05:38:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 4.0 MB/s 
     |████████████████████████████████| 81 kB 12.4 MB/s 
     |████████████████████████████████| 209 kB 62.7 MB/s 
     |████████████████████████████████| 78 kB 8.7 MB/s 
     |████████████████████████████████| 112 kB 65.9 MB/s 
     |████████████████████████████████| 49 kB 8.0 MB/s 
     |████████████████████████████████| 147 kB 59.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=899ffa5a0902f92be384a8cf95f2c130fc0ea14df895ae82ee792175f9e423af
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna

In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/shrisrivas/google_colab_datasets/main/Admission_Prediction.csv') 

In [ ]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [ ]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [ ]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [ ]:
x= df.drop(columns=['Serial No.' ,'Chance of Admit' ])
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.000000,118.0,4.0,4.5,4.5,9.65,1
1,324.000000,107.0,4.0,4.0,4.5,8.87,1
2,316.558763,104.0,3.0,3.0,3.5,8.00,1
3,322.000000,110.0,3.0,3.5,2.5,8.67,1
4,314.000000,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.000000,108.0,5.0,4.5,4.0,9.02,1
496,337.000000,117.0,5.0,5.0,5.0,9.87,1
497,330.000000,120.0,5.0,4.5,5.0,9.56,1
498,312.000000,103.0,4.0,4.0,5.0,8.43,0


In [ ]:
x.describe()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
count,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000
mean,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000
std,11.103952,6.051338,1.128802,0.991004,0.92545,0.604813,0.496884
min,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000
25%,309.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000
50%,316.558763,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000
75%,324.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000
max,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000


In [ ]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [ ]:
y = df['Chance of Admit' ]
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [ ]:
from sklearn.preprocessing import StandardScaler
std_sca = StandardScaler()
x   = std_sca.fit_transform(x)

In [ ]:
x

array([[ 1.84274116e+00,  1.78854223e+00,  7.78905651e-01, ...,
         1.09894429e+00,  1.77680627e+00,  8.86405260e-01],
       [ 6.70814288e-01, -3.10581135e-02,  7.78905651e-01, ...,
         1.09894429e+00,  4.85859428e-01,  8.86405260e-01],
       [ 5.12433309e-15, -5.27312752e-01, -1.07876604e-01, ...,
         1.73062093e-02, -9.54042814e-01,  8.86405260e-01],
       ...,
       [ 1.21170361e+00,  2.11937866e+00,  1.66568791e+00, ...,
         1.63976333e+00,  1.62785086e+00,  8.86405260e-01],
       [-4.10964364e-01, -6.92730965e-01,  7.78905651e-01, ...,
         1.63976333e+00, -2.42366993e-01, -1.12815215e+00],
       [ 9.41258951e-01,  9.61451165e-01,  7.78905651e-01, ...,
         1.09894429e+00,  7.67219636e-01, -1.12815215e+00]])

In [ ]:
from sklearn.metrics import mean_squared_error
def objective(trail , data = x , target = y ):
    train_x , test_x ,train_y , test_y = train_test_split(data , target , test_size = .10,random_state = 30  )
    param = {
        'tree_method':'gpu_hist',
        'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
        'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
        'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
        'n_estimator' :30000,
        'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
        'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
    }
    xgb_reg_model = xgb.XGBRegressor(**param)
    xgb_reg_model.fit(train_x,train_y,eval_set = [(test_x,test_y)] , verbose = True )
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y,pred_xgb)
    return rmse

In [ ]:
x

array([[ 1.84274116e+00,  1.78854223e+00,  7.78905651e-01, ...,
         1.09894429e+00,  1.77680627e+00,  8.86405260e-01],
       [ 6.70814288e-01, -3.10581135e-02,  7.78905651e-01, ...,
         1.09894429e+00,  4.85859428e-01,  8.86405260e-01],
       [ 5.12433309e-15, -5.27312752e-01, -1.07876604e-01, ...,
         1.73062093e-02, -9.54042814e-01,  8.86405260e-01],
       ...,
       [ 1.21170361e+00,  2.11937866e+00,  1.66568791e+00, ...,
         1.63976333e+00,  1.62785086e+00,  8.86405260e-01],
       [-4.10964364e-01, -6.92730965e-01,  7.78905651e-01, ...,
         1.63976333e+00, -2.42366993e-01, -1.12815215e+00],
       [ 9.41258951e-01,  9.61451165e-01,  7.78905651e-01, ...,
         1.09894429e+00,  7.67219636e-01, -1.12815215e+00]])

In [ ]:
find_param = optuna.create_study(direction='minimize')
find_param.optimize(objective,n_trials = 10)
find_param.best_trial.params

[I 2022-09-15 05:49:54,698] A new study created in memory with name: no-name-eb335e3a-f028-4e0a-9ded-1f61fd160798
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  import sys


[05:49:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.23292
[1]	validation_0-rmse:0.230832
[2]	validation_0-rmse:0.228656
[3]	validation_0-rmse:0.226642
[4]	validation_0-rmse:0.22467
[5]	validation_0-rmse:0.222755
[6]	validation_0-rmse:0.220846
[7]	validation_0-rmse:0.219078
[8]	validation_0-rmse:0.217328
[9]	validation_0-rmse:0.215442
[10]	validation_0-rmse:0.213636
[11]	validation_0-rmse:0.211933
[12]	validation_0-rmse:0.210079
[13]	validation_0-rmse:0.208316
[14]	validation_0-rmse:0.206556
[15]	validation_0-rmse:0.204888
[16]	validation_0-rmse:0.203176
[17]	validation_0-rmse:0.201334
[18]	validation_0-rmse:0.199741
[19]	validation_0-rmse:0.19819
[20]	validation_0-rmse:0.196661
[21]	validation_0-rmse:0.195025
[22]	validation_0-rmse:0.193394
[23]	validation_0-rmse:0.191922
[24]	validation_0-rmse:0.190319
[25]	validation_0-rmse:0.188784
[26]	validation_0-rmse:0.187371
[27]	validation_0-rmse

[I 2022-09-15 05:49:55,913] Trial 0 finished with value: 0.013157713413741851 and parameters: {'lambda': 0.8845265403567631, 'alpha': 4.357333587991134, 'colsample_bytree': 0.2, 'subsample': 0.9, 'learning_rate': 0.01, 'max_depth': 9, 'random_state': 243123, 'min_child_weight': 121}. Best is trial 0 with value: 0.013157713413741851.


[05:49:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.234934
[1]	validation_0-rmse:0.234886
[2]	validation_0-rmse:0.234837
[3]	validation_0-rmse:0.234782
[4]	validation_0-rmse:0.234719
[5]	validation_0-rmse:0.234674
[6]	validation_0-rmse:0.234627
[7]	validation_0-rmse:0.234566
[8]	validation_0-rmse:0.234503
[9]	validation_0-rmse:0.234454
[10]	validation_0-rmse:0.234408
[11]	validation_0-rmse:0.234341
[12]	validation_0-rmse:0.234291
[13]	validation_0-rmse:0.234243
[14]	validation_0-rmse:0.234182
[15]	validation_0-rmse:0.234135
[16]	validation_0-rmse:0.234091
[17]	validation_0-rmse:0.234029
[18]	validation_0-rmse:0.233964
[19]	validation_0-rmse:0.233904
[20]	validation_0-rmse:0.233829
[21]	validation_0-rmse:0.233782
[22]	validation_0-rmse:0.233735
[23]	validation_0-rmse:0.233671
[24]	validation_0-rmse:0.233602
[25]	validation_0-rmse:0.233553
[26]	validation_0-rmse:0.233485
[27]	validation_0-r

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  import sys


...

[05:50:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.233148
[1]	validation_0-rmse:0.231332
[2]	validation_0-rmse:0.229229
[3]	validation_0-rmse:0.227465
[4]	validation_0-rmse:0.225727
[5]	validation_0-rmse:0.224256
[6]	validation_0-rmse:0.222559
[7]	validation_0-rmse:0.220523
[8]	validation_0-rmse:0.218876
[9]	validation_0-rmse:0.217253
[10]	validation_0-rmse:0.215654
[11]	validation_0-rmse:0.214077
[12]	validation_0-rmse:0.212119
[13]	validation_0-rmse:0.210592
[14]	validation_0-rmse:0.209087
[15]	validation_0-rmse:0.207782
[16]	validation_0-rmse:0.206317
[17]	validation_0-rmse:0.204663
[18]	validation_0-rmse:0.203473
[19]	validation_0-rmse:0.202068
[20]	validation_0-rmse:0.200685
[21]	validation_0-rmse:0.199322
[22]	validation_0-rmse:0.19798
[23]	validation_0-rmse:0.196659
[24]	validation_0-rmse:0.195358
[25]	validation_0-rmse:0.194078
[26]	validation_0-rmse:0.192305
[27]	validation_0-rm

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  import sys


[34]	validation_0-rmse:0.183081
[35]	validation_0-rmse:0.18199
[36]	validation_0-rmse:0.180917
[37]	validation_0-rmse:0.179862
[38]	validation_0-rmse:0.178825
[39]	validation_0-rmse:0.177805
[40]	validation_0-rmse:0.176802
[41]	validation_0-rmse:0.175816
[42]	validation_0-rmse:0.174847
[43]	validation_0-rmse:0.173895
[44]	validation_0-rmse:0.172959
[45]	validation_0-rmse:0.172039
[46]	validation_0-rmse:0.171136
[47]	validation_0-rmse:0.170248
[48]	validation_0-rmse:0.169376
[49]	validation_0-rmse:0.168637
[50]	validation_0-rmse:0.167791
[51]	validation_0-rmse:0.16696
[52]	validation_0-rmse:0.166144
[53]	validation_0-rmse:0.165343
[54]	validation_0-rmse:0.164556
[55]	validation_0-rmse:0.163784
[56]	validation_0-rmse:0.163112
[57]	validation_0-rmse:0.162364
[58]	validation_0-rmse:0.16163
[59]	validation_0-rmse:0.16091
[60]	validation_0-rmse:0.160204
[61]	validation_0-rmse:0.15951
[62]	validation_0-rmse:0.15883
[63]	validation_0-rmse:0.158163
[64]	validation_0-rmse:0.157509
[65]	validatio

[I 2022-09-15 05:50:01,101] Trial 9 finished with value: 0.019681176330218212 and parameters: {'lambda': 2.17145287655763, 'alpha': 1.8265394148516374, 'colsample_bytree': 0.2, 'subsample': 0.3, 'learning_rate': 0.01, 'max_depth': 9, 'random_state': 30, 'min_child_weight': 88}. Best is trial 6 with value: 0.004344478140160474.


{'lambda': 0.0004539482499009905,
 'alpha': 8.468211589181221,
 'colsample_bytree': 0.4,
 'subsample': 0.4,
 'learning_rate': 1,
 'max_depth': 6,
 'random_state': 3454,
 'min_child_weight': 51}

In [ ]:
xgb.XGBRFRegressor()

XGBRFRegressor()

In [ ]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.103952,6.051338,1.128802,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,309.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,316.558763,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,324.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [ ]:
find_param.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.013158,2022-09-15 05:49:54.704021,2022-09-15 05:49:55.913462,0 days 00:00:01.209441,4.357334,0.2,0.884527,0.0100,9,121,243123,0.9,COMPLETE
1,1,0.052607,2022-09-15 05:49:55.915296,2022-09-15 05:49:56.468456,0 days 00:00:00.553160,0.547904,1.0,0.005775,0.0003,9,23,20,0.1,COMPLETE
2,2,0.010756,2022-09-15 05:49:56.470295,2022-09-15 05:49:57.022323,0 days 00:00:00.552028,0.000137,1.0,6.990662,0.0100,5,77,20,1.0,COMPLETE
3,3,0.011953,2022-09-15 05:49:57.023905,2022-09-15 05:49:57.615106,0 days 00:00:00.591201,0.006598,0.4,0.126379,0.0100,12,81,2000,0.8,COMPLETE
4,4,0.052344,2022-09-15 05:49:57.619107,2022-09-15 05:49:58.171087,0 days 00:00:00.551980,0.000872,0.5,0.000189,0.0003,8,119,2000,0.6,COMPLETE
5,5,0.007418,2022-09-15 05:49:58.174745,2022-09-15 05:49:58.784401,0 days 00:00:00.609656,9.055513,1.0,5.133951,1.0000,5,24,3454,1.0,COMPLETE
6,6,0.004344,2022-09-15 05:49:58.786184,2022-09-15 05:49:59.372402,0 days 00:00:00.586218,8.468212,0.4,0.000454,1.0000,6,51,3454,0.4,COMPLETE
7,7,0.012519,2022-09-15 05:49:59.374659,2022-09-15 05:49:59.945682,0 days 00:00:00.571023,0.002993,0.2,0.012143,0.0100,5,91,30,0.9,COMPLETE
8,8,0.013569,2022-09-15 05:49:59.947993,2022-09-15 05:50:00.543478,0 days 00:00:00.595485,0.003233,0.8,0.961704,0.0100,4,150,3454,0.7,COMPLETE
9,9,0.019681,2022-09-15 05:50:00.545094,2022-09-15 05:50:01.100761,0 days 00:00:00.555667,1.826539,0.2,2.171453,0.0100,9,88,30,0.3,COMPLETE


In [ ]:
optuna.visualization.plot_optimization_history(find_param)

In [ ]:
optuna.visualization.plot_slice(find_param)

In [ ]:
optuna.visualization.plot_contour(find_param , params = ['alpha' , 'lambda'])

In [ ]:
best_param = {'lambda': 0.5646022891371127,
 'alpha': 0.35848071012137483,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 30}

In [ ]:
xgb_final_model = xgb.XGBRegressor(**best_param)
xgb_final_model.fit(x,y)

[05:52:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=0.35848071012137483, colsample_bytree=0.9,
             lambda=0.5646022891371127, learning_rate=1, max_depth=11,
             min_child_weight=30, random_state=30, subsample=0.2)

In [ ]:
xgb_final_model.score(x,y)

0.7548698565043936

In [ ]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split